## 11. TLS handshake in one paragraph

Transport Layer Security (TLS) wraps a TCP connection in **encryption + authentication**.

Handshake flow (TLS 1.2 simplified):
1. **ClientHello** – lists cipher suites & random.
2. **ServerHello** – picks cipher, sends certificate (public key).
3. **Key exchange** – client encrypts a *pre‑master secret* with server’s public key.
4. Both derive the same session keys → **Finished** messages verified.

STARTTLS upgrades an existing plaintext port (SMTP 25 → TLS). *Pure TLS* ports (e.g., HTTPS 443)
start encrypted from byte 0.

```python
import ssl, socket
ctx = ssl.create_default_context()
with ctx.wrap_socket(socket.socket(), server_hostname='example.com') as s:
    s.connect(('example.com', 443))
    print('TLS version', s.version())
```

### Quick check

1. True / False TLS works on top of UDP.

2. Port 587 (mail submission) typically uses:
  a. pure TLS   b. STARTTLS upgrade

<details><summary>Answer key</summary>

1. **False** – TLS assumes reliable stream (TCP).
2. **b**.

</details>

## 12. From raw bytes to protocols: framing

Sockets are byte streams; you must define **message boundaries**.

* **Length‑prefix** – first 4‑byte int gives payload size.
* **Delimiter** – e.g., `\r\n` ends a line (SMTP, HTTP headers).
* **Fixed‑size records** – always 512 bytes (DNS over UDP).

Wrong framing ⇒ sender/receiver fall out of sync, producing corrupted data or hangs.

```python
import struct, socket
msg = b'hello'
payload = struct.pack('!I', len(msg)) + msg  # length‑prefixed
print(payload)
```

### Quick check

1. Using `\0` as terminator is safe for binary data?
  a. yes   b. no

2. True / False Length‑prefix lets receiver allocate exact buffer size.

<details><summary>Answer key</summary>

1. **b** – null byte may appear in binary.
2. **True**.

</details>

## 13. HTTP request anatomy

HTTP 1.1 request layout:
```
GET /path?x=1 HTTP/1.1  ← request line
Host: example.com        ← headers
User-Agent: curl/8.5
                         ← blank line
{ "json": true }       ← optional body
```
Headers are ASCII key–value pairs ending with CR‑LF. Path may include query string; body length signalled by `Content-Length` or `Transfer-Encoding: chunked`.

```bash
curl -v -X POST https://httpbin.org/post -H 'Content-Type: application/json' \
     -d '{"hi":true}'
```

### Quick check

1. Missing blank line before body will cause server to:
  a. parse body as header   b. accept anyway

2. True / False HTTP headers are case‑insensitive.

<details><summary>Answer key</summary>

1. **a**.
2. **True**.

</details>

## 14. HTTP methods: GET, POST, PUT, PATCH, DELETE

* **GET** – safe, idempotent, no body semantics (though body allowed by spec). Use for reads.
* **POST** – non‑idempotent create/execute.
* **PUT** – full replace; idempotent.
* **PATCH** – partial update.
* **DELETE** – idempotent removal.

**Idempotent** = same call repeated → same end state. Important for retries.

```python
import requests
r=requests.put('https://httpbin.org/put', json={'x':1})
print(r.status_code)
```

### Quick check

1. Which method is NOT idempotent by spec?
  a. PUT   b. POST

2. True / False GET requests should change server state.

<details><summary>Answer key</summary>

1. **b**.
2. **False**.

</details>

## 15. Status code classes & common meanings

| Class | Meaning |
|-------|---------|
| 1xx | informational, rarely seen |
| 2xx | success (200 OK, 201 Created, 204 No Content) |
| 3xx | redirects (301 permanent, 302 temporary) |
| 4xx | client error (400 Bad Request, 401 Unauthorized, 403 Forbidden, 404 Not Found) |
| 5xx | server error (500 Internal Server Error, 503 Service Unavailable) |

Rule of thumb: **Anything 4xx is caller’s fault; 5xx is server’s fault.**

```bash
curl -i https://httpbin.org/status/404
```

### Quick check

1. `204` response contains body?
  a. yes   b. no

2. True / False `301` should include `Location` header.

<details><summary>Answer key</summary>

1. **b** – no payload.
2. **True**.

</details>